In [ ]:
import warnings
warnings.filterwarnings('ignore')
import json
from pymongo import MongoClient, GEO2D

In [ ]:
zips_list = []
for line in open('zips.json', 'r'):
    zips_list.append(json.loads(line))

In [ ]:
myclient = MongoClient("mongodb://localhost:27017/")

#use database named "organisation"
mydb = myclient["HW2"]

mycol = mydb['hw2_zips']

In [ ]:
mydb.hw2_zips.create_index([("loc", GEO2D)])

In [ ]:
myclient.list_database_names()

In [ ]:
mydb.list_collection_names()

In [ ]:
list(mydb.hw2_zips.find_one().keys())

In [ ]:
mycol.insert_many(zips_list)

In [ ]:
mydb.hw2_zips.drop()

## Q1
Count the total number of cities in Washington state (code: "WA").

In [ ]:
Q1_pipeline_match = {"$match": {"state": "WA"}}

Q1_doc = mycol.aggregate([Q1_pipeline_match])
print('The total number of cities in Washington state is {}.'.format(len(list(Q1_doc))))

## Q2
Find the total population of each state (i.e., sort states by their population in the ascending
order).

In [ ]:
Q2_pipe_sum = {'$group': {'_id': '$state', 'total': {'$sum': '$pop'}}}
Q2_pipe_sort = { '$sort' : { 'total' : 1 } }

Q2_sum = mycol.aggregate([Q2_pipe_sum,Q2_pipe_sort])

for document in Q2_sum:
    print('The populaton of {} is {}.'.format(document['_id'],document['total']))

## Q3
Find the 10 closest cities to WEST BROOKLYN, IL. You might want to use the "$near" operator.

In [ ]:
Q3_pip_WB_IL =  {"$match": {"state": "IL",'city':'WEST BROOKLYN'}}

Q3_WB_IL = mycol.aggregate([Q3_pip_WB_IL])

Q3_WB_IL_coord = list(Q3_WB_IL)[0]['loc']

Q3_list = 'The 10 closest cities to WEST BROOKLYN, IL is: '
for doc in mycol.find({"loc": {"$near": Q3_WB_IL_coord}}).limit(11):
    if doc['city'] != 'WEST BROOKLYN':
        Q3_list = Q3_list+doc['city'] + ', '
print(Q3_list[:-2]+'.')

## Q4
Considering the `region` of each US state, according to this source, find the total population in
each of the four regions (West, South, Midwest, and Northeast).

In [ ]:
Northeast = ['CT','ME','MA','NH','RI','VT','NJ','NY','PA']

West = ['AZ','CO','ID','NM','MT','UT','NV','WY','AK','CA','HI','OR','WA']

Midwest = ['IN','IL','MI','OH','WI','IA''NE''KS','ND','MN','SD''MS']

South = ['DE''DC','FL','GA','MD','NC''SC','VA','WV','AL','KY','MS','TN','AR','LA','OK','TX']

Q4_pipe_region=  {
    '$project': {
        '_id':'$_id',
        'pop':'$pop',
        'West' : {'$in': [ '$state',West]},
        'Northeast' : {'$in': [ '$state',Northeast]},
        'Midwest' : {'$in': [ '$state',Midwest]},
        'South' : {'$in': [ '$state',South]}
    }
}
Q4_pipe_northeast =  { '$match' : { 'Northeast' : True } }
Q4_pipe_west =  { '$match' : { 'West' : True } }
Q4_pipe_midwest =  { '$match' : { 'Midwest' : True } }
Q4_pipe_south =  { '$match' : { 'South' : True } }
Q4_pipe_sum =  {'$group':{'_id':None,'Total Population': { '$sum': "$pop" }}}

print('The population in Northeast is {}.'.format(list(mycol.aggregate([Q4_pipe_region,Q4_pipe_sum_northeast,Q4_pipe_sum]))[0]['Total Population']))
print('The population in West is {}.'.format(list(mycol.aggregate([Q4_pipe_region,Q4_pipe_west,Q4_pipe_sum]))[0]['Total Population']))
print('The population in Midwest is {}.'.format(list(mycol.aggregate([Q4_pipe_region,Q4_pipe_midwest,Q4_pipe_sum]))[0]['Total Population']))
print('The population in South is {}.'.format(list(mycol.aggregate([Q4_pipe_region,Q4_pipe_south,Q4_pipe_sum]))[0]['Total Population']))

## Q5
Find the 3 most populated cities for each state (in one query).

In [ ]:
Q5_Pipeline_city_pop = { '$group':{'_id': { 'state': "$state", 'city': "$city" },'pop': { '$sum': "$pop" }}}
Q5_Pipline_sort = { '$sort': { 'pop': 1 } }


Q5_pipe_list = {'$group': {'_id':"$_id.state", 'pop': {'$push': '$pop'},'city': {'$push': '$_id.city'}}}

Q5_pipe_select = {
    '$project': {'_id': 1, 
                 'city_1': {'$arrayElemAt': ['$city', -1]},
                 'city_2': {'$arrayElemAt': ['$city', -2]},
                 'city_3': {'$arrayElemAt': ['$city', -3]},
                 'pop_1': {'$arrayElemAt': ['$pop', -1]},
                 'pop_2': {'$arrayElemAt': ['$pop', -2]},
                 'pop_3': {'$arrayElemAt': ['$pop', -3]}
                }
}

Q5_3_most = mycol.aggregate([Q5_Pipeline_city_pop,Q5_Pipline_sort,Q5_pipe_list,Q5_pipe_select])

for state in list(Q5_3_most):
    if(len(state)) ==7:
        print('The first, second, third most populated cities in {} are {},{} and {}, with population of {},{} and {}.'.format(state['_id'],state['city_1'],state['city_2'],state['city_3'],state['pop_1'],state['pop_2'],state['pop_3']))
    else:
        print('The first and second most populated cities in {} are {} and {}, with population of {} and {}.'.format(state['_id'],state['city_1'],state['city_2'],state['pop_1'],state['pop_2']))        
        